# Visualization tutorial 🎨

This notebook shows the visualization tools of `airo-drake`.

## 1. Building the Scene 🏗️

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import add_manipulator, add_floor, add_meshcat, finish_build
from airo_drake import SingleArmScene


robot_diagram_builder = RobotDiagramBuilder() 

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur5e", "robotiq_2f_85")
add_floor(robot_diagram_builder)

robot_diagram, context = finish_build(robot_diagram_builder, meshcat)
del robot_diagram_builder # no longer needed

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)
scene

In [ ]:
joints = np.deg2rad([0, -45, -90, -90, 90, 0])

plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, scene.arm_index, joints)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

In [ ]:
plant.GetPositions(plant_context, scene.gripper_index)

🧙‍♂️ Closing the gripper is a bit more tricky because the mimic joints are not supported in Drake: 

In [ ]:
v = 0.81 # magic joint value for to close the gripper
plant.SetPositions(plant_context, scene.gripper_index, np.array([v, -v, v] * 2))
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

## 2. Visualizing frames 🖼️

⏩ Let's start by calculating the forward kinematics of the robot and visualizing the TCP frame:

In [ ]:
from ur_analytic_ik import ur5e

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175 # 175 mm in z

tcp_pose = ur5e.forward_kinematics(*joints) @ tcp_transform

with np.printoptions(precision=3, suppress=True):
    print(tcp_pose)

In [ ]:
from airo_drake import visualize_frame

visualize_frame(scene.meshcat, tcp_pose, "tcp_frame")

In [ ]:
from pydrake.math import RigidTransform

visualize_frame(scene.meshcat, RigidTransform(), "world_frame", length=1.0, opacity=0.2)

In [ ]:
scene.meshcat.Delete("tcp_frame")

## 3. Visualizing IK solutions 🦾

## 4. Visualizing paths 🛤️

## 5. Visualizing trajectories 🌠